<a href="https://colab.research.google.com/github/developerabhi14/ML-Notebooks/blob/main/FineTuning_BERT_for_Toxic_Comment_Detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
! pip install transformers

In [2]:
import pandas as pd

data=pd.read_csv("train.csv",encoding="utf-8")
data.head()

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,0000997932d777bf,Explanation\nWhy the edits made under my usern...,0,0,0,0,0,0
1,000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0
2,000113f07ec002fd,"Hey man, I'm really not trying to edit war. It...",0,0,0,0,0,0
3,0001b41b1c6bb37e,"""\nMore\nI can't make any real suggestions on ...",0,0,0,0,0,0
4,0001d958c54c6e35,"You, sir, are my hero. Any chance you remember...",0,0,0,0,0,0


In [3]:
data['toxic'].value_counts()

,count
toxic,
0,144277
1,15294


In [4]:
data=data[['comment_text', 'toxic']]
data=data[0:1000]
data.head()

,comment_text,toxic
0,Explanation\nWhy the edits made under my usern...,0
1,D'aww! He matches this background colour I'm s...,0
2,"Hey man, I'm really not trying to edit war. It...",0
3,"""\nMore\nI can't make any real suggestions on ...",0
4,"You, sir, are my hero. Any chance you remember...",0


In [5]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score
import torch
from transformers import TrainingArguments, Trainer
from transformers import BertTokenizer, BertForSequenceClassification

In [6]:
tokenizer=BertTokenizer.from_pretrained('bert-base-uncased')
model=BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=2)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [7]:
model=model.to('cuda')
sample_data=['I am eating apple', 'I am playing cricket']
tokenizer(sample_data, padding=True, truncation=True, max_length=512)

{'input_ids': [[101, 1045, 2572, 5983, 6207, 102], [101, 1045, 2572, 2652, 4533, 102]], 'token_type_ids': [[0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0]], 'attention_mask': [[1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1]]}

In [8]:
X=list(data['comment_text'])
y=list(data['toxic'])

X_train, X_val, y_train, y_val=train_test_split(X,y, test_size=0.2, stratify=y)
X_train_tokenized=tokenizer(X_train, padding=True, truncation=True, max_length=512)
X_val_tokenized=tokenizer(X_val, padding=True, truncation=True, max_length=512)

In [9]:
X_train_tokenized.keys()

dict_keys(['input_ids', 'token_type_ids', 'attention_mask'])

In [10]:
print(X_train_tokenized['input_ids'][0])

[101, 1000, 1052, 1012, 1055, 1012, 2009, 1005, 1055, 2025, 13205, 2000, 2831, 2000, 2111, 2369, 2037, 10457, 1010, 3531, 6366, 2115, 7928, 2013, 2720, 8458, 1005, 1055, 2831, 3931, 1012, 14461, 2017, 1005, 2128, 2157, 1025, 1045, 2253, 2000, 4638, 2115, 3025, 10086, 1998, 2179, 1037, 3931, 2006, 1996, 8348, 2609, 2008, 11479, 2009, 1000, 1000, 18220, 1000, 1000, 1010, 2021, 2085, 1045, 2572, 4531, 2116, 2062, 2008, 6297, 2009, 11178, 1012, 4283, 2005, 1996, 10086, 2015, 1012, 1006, 100, 1007, 1000, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0

In [11]:
len(X_train), len(X_val)

(800, 200)

In [12]:
class Dataset(torch.utils.data.Dataset):
  def __init__(self, encodings, labels=None):
    self.encodings=encodings
    self.labels=labels

  def __getitem__(self, idx):
    item={key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
    if self.labels:
      item['labels']=torch.tensor(self.labels[idx])
    return item

  def __len__(self):
    return len(self.encodings['input_ids'])

In [13]:
train_dataset=Dataset(X_train_tokenized, y_train)
val_dataset=Dataset(X_val_tokenized, y_val)

In [14]:
train_dataset[5]

{'input_ids': tensor([  101,  3492,  2172,  3071,  2013,  6031,  2221,  1013,  4193,  4655,
          2001,  2141,  2012,  8904,  2015,  4212,  2902,  1012,  2870,  2443,
          1012,  2174,  1010,  1045,  1005,  1049,  2025,  2469,  2023, 24209,
         11475, 14213,  3087,  2004,  2108,  1037,  8904,  2015,  4212,  3128,
          1012,  5586,  4097,  2003,  1010,  1045,  2903,  1010,  2941,  2013,
          1996,  2237,  1997,  2697, 26214, 11795,  2063,  1012,  1517,  1996,
         11003, 27121,  7615,  2001,  2794,  2011,  3963,  1012,  2531,  1012,
         22777,  1012, 16666,  5840,  1024,  2654,  1024,  5401,  1010,  2257,
          2539,  1010,  2289,  1006, 11396,  1007,   102,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,   

In [15]:
def compute_metrics(p):
  print(type(p))
  pred,labels=p
  pred=np.argmax(pred, axis=1)

  accuracy=accuracy_score(y_true=labels, y_pred=pred)
  recall=recall_score(y_true=labels, y_pred=pred)
  precision=precision_score(y_true=labels, y_pred=pred)
  f1=f1_score(y_true=labels, y_pred=pred)

  return {"accuracy":accuracy, "precision":precision, "recall":recall, "f1":f1}

In [16]:
# Define Trainer

args=TrainingArguments(
    output_dir='output',
    num_train_epochs=5,
    per_device_train_batch_size=8
)

trainer=Trainer(
    model=model, args=args,train_dataset=train_dataset, eval_dataset=val_dataset, compute_metrics=compute_metrics
)

In [17]:
trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: avskh to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Step,Training Loss
500,0.094100


TrainOutput(global_step=500, training_loss=0.09407918548583985, metrics={'train_runtime': 447.626, 'train_samples_per_second': 8.936, 'train_steps_per_second': 1.117, 'total_flos': 1052444221440000.0, 'train_loss': 0.09407918548583985, 'epoch': 5.0})

In [18]:
trainer.evaluate()

<class 'transformers.trainer_utils.EvalPrediction'>


{'eval_loss': 0.2521352469921112,
 'eval_accuracy': 0.96,
 'eval_precision': 0.8823529411764706,
 'eval_recall': 0.7142857142857143,
 'eval_f1': 0.7894736842105263,
 'eval_runtime': 6.5901,
 'eval_samples_per_second': 30.349,
 'eval_steps_per_second': 3.794,
 'epoch': 5.0}

In [24]:
text="that was a good point"
input=tokenizer(text, padding=True, truncation=True, return_tensors='pt').to('cuda')
outputs=model(**input)
predictions=torch.nn.functional.softmax(outputs.logits,dim=-1)
predictions=predictions.cpu().detach().numpy()
predictions

array([[9.9977165e-01, 2.2833810e-04]], dtype=float32)

In [22]:
trainer.save_model('CustomModel')

In [23]:
model_2=BertForSequenceClassification.from_pretrained('CustomModel')
model_2.to('cuda')

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e

In [29]:
text="you rock"
input=tokenizer(text, padding=True, truncation=True, return_tensors='pt').to('cuda')
outputs=model_2(**input)
predictions=torch.nn.functional.softmax(outputs.logits,dim=-1)
predictions=predictions.cpu().detach().numpy()
predictions

array([[0.9039177 , 0.09608237]], dtype=float32)